In [ ]:
from operator import index
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
import os
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000

# ✅ 브라우저 초기화 및 로그인
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# ✅ 로그인 함수
def login_to_console(driver):
    try:
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        pyperclip.copy("dksrufp0607@naver.com")
        id_input.send_keys(Keys.CONTROL, 'v')

        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "password"))
        )
        pyperclip.copy("ghkfkd159!")
        pw_input.send_keys(Keys.CONTROL, 'v')
        time.sleep(1)
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]'))
        )
        login_button.click()
        print("✅ 로그인 완료!")
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ 로그인 실패: {e}")

# ✅ 콘솔 페이지 이동
def navigate_to_page(driver):
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    print("✅ 페이지 로드 완료!")

# ✅ "확인" 버튼 클릭
def confirm_click(driver):
    try:
        print("🔍 '확인' 버튼 탐색 중...")
        confirm_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]'))
        )
        confirm_button.click()
        print("✅ 확인 버튼 클릭 완료!")
    except TimeoutException:
        print("⚠️ '확인' 버튼을 찾지 못했습니다. 페이지가 제대로 로드되었는지 확인하세요.")
    except Exception as e:
        print(f"⚠️ 확인 버튼 클릭 실패: {e}")

# ✅ CSV 데이터 로드 함수 (검색용)
def get_csv_value(row, col):
    csv_path = "C:/엑셀/Ranking_1.csv"
    try:
        df = pd.read_csv(csv_path)
        if row < df.shape[0] and col < df.shape[1]:
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")
            return df.iloc[row, col]
        else:
            print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
            return None
    except FileNotFoundError:
        print(f"⚠️ CSV 파일을 찾을 수 없습니다: {csv_path}")
        return None
    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None

# ✅ 검색 실행 함수
def execute_tasks(driver, row, col):
    try:
        b2_value = get_csv_value(row, col)
        if not b2_value:
            print("⚠️ CSV에서 값을 가져오지 못했습니다. 작업을 중단합니다.")
            return

        table_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]'))
        )
        table_dropdown.click()
        print("✅ 테이블 드롭다운 클릭 완료!")

        base_data_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@role="option"][span[text()="BASE_DATA"]]'))
        )
        base_data_option.click()
        print("✅ BASE_DATA 선택 완료!")

        detail_search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]'))
        )
        time.sleep(3)
        try:
            detail_search_button.click()
        except ElementClickInterceptedException:
            print("⚠️ '상세 검색' 버튼 클릭 차단됨. JavaScript로 재시도...")
            driver.execute_script("arguments[0].click();", detail_search_button)
        print("✅ 상세 검색 클릭 완료!")

        search_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "defaultSearchValue"))
        )
        search_input.clear()
        search_input.send_keys(b2_value)
        print(f"✅ B2 값 입력 완료: {b2_value}")

        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]'))
        )
        search_button.click()
        print("✅ 검색 버튼 클릭 완료!")

        uuid_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(@class, "_link_a1mor_113")]'))
        )
        uuid_element.click()
        print("✅ UUID 클릭 완료!")
    except Exception as e:
        print(f"⚠️ 검색 작업 중 오류 발생: {e}")

# ✅ Ace Editor 값 가져오기 함수 (object 타입 전용)
def get_ace_editor_value(driver, ace_element):
    try:
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script("return typeof ace !== 'undefined' && ace.edit(arguments[0]) !== null", ace_element)
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", ace_element)
        value = driver.execute_script("return ace.edit(arguments[0]).getValue();", ace_element).strip()
        print(f"✅ Ace Editor 값 가져오기 완료: {value}")
        return value
    except Exception as e:
        print(f"⚠️ Ace Editor 값 가져오기 실패: {e}")
        return None

# ✅ 데이터 추출 및 저장 함수 (object 타입 전용)
def copy_and_save_data(driver, output_filename):
    try:
        # 수정 버튼 클릭
        edit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui mini primary button") and text()="수정"]'))
        )
        edit_button.click()
        print("✅ 수정 버튼 클릭 완료!")
        time.sleep(1)  # 모달 로딩 대기

        # 웹 화면 축소
        driver.execute_script("document.body.style.zoom='25%'")
        print("✅ 웹 화면 25% 축소 완료!")
        time.sleep(2)

        # iframe 또는 기본 컨텍스트 확인
        driver.switch_to.default_content()
        context_found = False
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
            )
            print("🔍 기본 콘텐츠에서 요소 발견!")
            context_found = True
        except TimeoutException:
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            for i, frame in enumerate(iframes):
                driver.switch_to.frame(frame)
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
                    )
                    print(f"🔍 iframe {i}에서 요소 발견!")
                    context_found = True
                    break
                except TimeoutException:
                    driver.switch_to.default_content()
            if not context_found:
                print("⚠️ 모든 컨텍스트에서 요소를 찾지 못함")
                return

        # 데이터 추출 (object 타입만)
        data = []
        attempt = 0
        max_attempts = 3
        while attempt < max_attempts:
            try:
                elements = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
                )
                found = False
                for idx, element in enumerate(elements):
                    print(f"🔍 현재 처리 중인 요소: {idx + 1}/{len(elements)}")
                    try:
                        # 데이터 타입 확인
                        try:
                            data_type = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(2) p").text.strip()
                        except NoSuchElementException:
                            data_type = "unknown"
                            print(f"⚠️ 데이터 타입을 찾을 수 없음, 'unknown'으로 설정")

                        # object 타입인 경우에만 처리
                        if data_type == "object":
                            try:
                                col_name = element.find_element(By.CSS_SELECTOR, 'input[type="text"][name$=".columnName"]').get_attribute("value")
                            except NoSuchElementException:
                                col_name = "Unknown"
                                print(f"⚠️ {col_name} 컬럼명을 찾을 수 없음")

                            try:
                                ace_editor = element.find_element(By.CSS_SELECTOR, "div.ace_editor")
                                driver.execute_script("arguments[0].click();", ace_editor)
                                time.sleep(1)  # 포커스 후 대기
                                value = get_ace_editor_value(driver, ace_editor)
                            except NoSuchElementException:
                                try:
                                    value_field = element.find_element(By.CSS_SELECTOR, 'textarea[name$=".dataValue"]')
                                    value = value_field.get_attribute("value")
                                except NoSuchElementException:
                                    value = "N/A"
                                    print(f"⚠️ {col_name} 값 필드를 찾을 수 없음")
                                time.sleep(1)

                            data.append([col_name, data_type, value])
                            print(f"✅ {col_name} 데이터 추출 완료: {value} (타입: {data_type})")
                            found = True

                    except StaleElementReferenceException:
                        print(f"⚠️ StaleElement 발생, 다음 요소로 이동")
                        continue
                if found:
                    break
            except TimeoutException:
                attempt += 1
                print(f"⚠️ 요소 탐색 실패 (시도 {attempt}/{max_attempts})")
                if attempt == max_attempts:
                    print("⚠️ 최종 탐색 실패")
            except Exception as e:
                print(f"⚠️ 오류 발생: {e}")
                break

        # 엑셀에 저장
        driver.switch_to.default_content()
        if data:
            df = pd.DataFrame(data, columns=["Column Name", "Data Type", "Value"])
            df.to_excel(output_filename, index=False, engine="openpyxl")
            print(f"✅ 데이터가 {output_filename}에 저장되었습니다!")
        else:
            print("⚠️ 추출된 데이터가 없습니다.")

        # 모달 닫기 (취소 버튼)
        cancel_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "취소")]'))
        )
        cancel_button.click()
        print("✅ 모달 닫기 완료!")

    except Exception as e:
        print(f"⚠️ 데이터 추출 및 저장 실패: {e}")
        driver.switch_to.default_content()

# ✅ 메인 실행
if __name__ == "__main__":
    output_filename = "C:\\JJG\\Start\\sjdj\\3300_output.xlsx"  # 저장할 파일 경로
    print(f"🔍 현재 작업 디렉토리: {os.getcwd()}")

    driver1 = setup_browser(0)
    try:
        login_to_console(driver1)
        navigate_to_page(driver1)
        confirm_click(driver1)
        execute_tasks(driver1, 0, 5)
        copy_and_save_data(driver1, output_filename)
    except KeyboardInterrupt:
        print("⚠️ 사용자가 프로그램을 중단했습니다.")
    except Exception as e:
        print(f"⚠️ 전체 실행 중 오류: {e}")
    finally:
        time.sleep(1)
        # driver1.quit()
        print("✅ 브라우저 종료")

🔍 현재 작업 디렉토리: c:\JJG\Start\sjdj
✅ 로그인 완료!
✅ 페이지 로드 완료!
🔍 '확인' 버튼 탐색 중...
✅ 확인 버튼 클릭 완료!
🔹 0행 3열 값 (출력용): 슈바니
✅ 테이블 드롭다운 클릭 완료!
✅ BASE_DATA 선택 완료!
✅ 상세 검색 클릭 완료!
✅ B2 값 입력 완료: 0831d8a0-c410-11ef-ac3c-67eaa6cff280
✅ 검색 버튼 클릭 완료!
✅ UUID 클릭 완료!
🔍 총 226개의 요소를 탐색합니다.
🔍 탐색 중: 인덱스 1/226
✅ 추출 완료 (인덱스 1): event8_Receive, number, 39
🔍 탐색 중: 인덱스 2/226
📋 Ace Editor 값 복사 (기본 컨텍스트): [
    10,
    0,
    0,
    3,
    0,
    0,
    0,
    0,
    0,
    0,
    0
]
✅ 추출 완료 (인덱스 2): purchaseCurMileage, object, [
    10,
    0,
    0,
    3,
    0,
    0,
    0,
    0,
    0,
    0,
    0
]
🔍 탐색 중: 인덱스 3/226
✅ 추출 완료 (인덱스 3): hpGrowth2_1, number, 400310
🔍 탐색 중: 인덱스 4/226
📋 Ace Editor 값 복사 (기본 컨텍스트): [
    20,
    0,
    0,
    3,
    0,
    0,
    0,
    0,
    0,
    0,
    0
]
✅ 추출 완료 (인덱스 4): mission_Weekly_Check, object, [
    20,
    0,
    0,
    3,
    0,
    0,
    0,
    0,
    0,
    0,
    0
]
🔍 탐색 중: 인덱스 5/226
📋 Ace Editor 값 복사 (기본 컨텍스트): [
    30,
    0,
    0,
    3,
    0,
    0,
    0,
    